### 1. 데이터셋 및 전처리

-   **데이터셋 구성**
    -   웹 스크레이핑을 통해 수집한 자체 데이터셋을 사용할 예정.
    -   클래스는 **Street, Minimal, Amekaji, Feminine** 총 4개.
    -   데이터는 클래스당 100장의 Full body 이미지, 총 400장으로 구성됨.

-   **전처리 및 데이터 증강 계획**
    -   모든 이미지는 사전 훈련 모델의 입력 크기인 `(224, 224)`로 표준화할 것.
    -   픽셀 값은 `[0, 1]` 범위로 정규화하여 학습 안정성을 확보할 예정.
    -   훈련 데이터에 한해 `ImageDataGenerator`를 활용, Data Augmentation을 적용하여 모델의 과적합을 방지하고 일반화 성능을 향상.

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator


DATA_DIR = '../image'
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,                 
    rotation_range=20,             
    width_shift_range=0.2,          
    height_shift_range=0.2,        
    shear_range=0.2,                
    zoom_range=0.2,                 
    horizontal_flip=True,           
    fill_mode='nearest',            
    validation_split=0.2            
)

validation_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

In [ ]:
# === 1. 데이터셋 및 전처리 (간단 버전) ===
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

DATA_DIR = '../image'
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
VAL_SPLIT = 0.2

# 폴더명과 동일한 클래스 순서 고정
CLASS_NAMES = ['Street', 'Minimal', 'Amekaji', 'Feminine']

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=VAL_SPLIT
)
valid_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=VAL_SPLIT
)

train_gen = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    classes=CLASS_NAMES,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    seed=42
)
valid_gen = valid_datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    classes=CLASS_NAMES,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

steps_per_epoch   = max(1, train_gen.samples // BATCH_SIZE)
validation_steps  = max(1, valid_gen.samples // BATCH_SIZE)
print("클래스 인덱스:", train_gen.class_indices)
print("학습/검증 샘플 수:", train_gen.samples, valid_gen.samples)


### 2. 모델 아키텍처 및 훈련

-   **모델 아키텍처 선정**
    -   전이 학습의 효과와 아키텍처별 성능 비교를 위해 **VGG16**과 **ResNet50**을 각각 베이스라인 및 비교 모델로 사용.
    -   두 모델 모두 ImageNet으로 사전 훈련된 **합성곱 베이스(Convolutional Base)의 가중치는 동결(Freeze)**하여 사용될 예정.

-   **미세조정(Fine-tuning) 설계**
    -   동결된 베이스 모델 위에 Classifier Head를 추가할 계획.
    -   분류기는 `GlobalAveragePooling2D`, `Dropout(0.5)`, `Dense(1024, 'relu')`, 그리고 최종 `Dense(4, 'softmax')` 층으로 구성됨. `Dropout`을 통해 과적합을 억제할 예정.

-   **훈련 계획**
    -   옵티마이저는 `Adam`, 손실 함수는 `categorical_crossentropy`를 사용할 것.
    -   훈련은 초기 20 에포크(Epoch) 동안 진행하며, 학습 곡선(정확도 및 손실)을 시각화하여 과적합 여부를 검토할 예정.


In [ ]:
# === 2. 모델 아키텍처 & 훈련 (간단 버전) ===
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16, ResNet50

NUM_CLASSES = len(CLASS_NAMES)

def build_model(arch='vgg16', input_shape=(224,224,3), num_classes=4, freeze_base=True):
    if arch == 'vgg16':
        base = VGG16(include_top=False, weights='imagenet', input_shape=input_shape)
    elif arch == 'resnet50':
        base = ResNet50(include_top=False, weights='imagenet', input_shape=input_shape)
    else:
        raise ValueError("arch는 'vgg16' 또는 'resnet50'만 허용")

    base.trainable = not freeze_base  # 간단: 기본은 동결(True)
    x = layers.Input(shape=input_shape)
    y = base(x, training=False)
    y = layers.GlobalAveragePooling2D()(y)
    y = layers.Dropout(0.5)(y)
    y = layers.Dense(1024, activation='relu')(y)
    out = layers.Dense(num_classes, activation='softmax')(y)
    model = models.Model(x, out)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-4),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

def train_once(arch, epochs=10):
    print(f"\n=== Training: {arch.upper()} ===")
    model = build_model(arch=arch, input_shape=(*IMAGE_SIZE,3), num_classes=NUM_CLASSES, freeze_base=True)
    es = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)
    hist = model.fit(
        train_gen,
        steps_per_epoch=steps_per_epoch,
        validation_data=valid_gen,
        validation_steps=validation_steps,
        epochs=epochs,
        callbacks=[es],
        verbose=1
    )
    # 간단 평가
    val_loss, val_acc = model.evaluate(valid_gen, steps=validation_steps, verbose=0)
    print(f"[{arch.upper()}] Val Acc = {val_acc:.4f}")

    # 학습곡선 간단 플롯(정확도만)
    plt.figure(figsize=(5,3))
    plt.plot(hist.history['accuracy'], label='train')
    plt.plot(hist.history['val_accuracy'], label='val')
    plt.title(f'{arch.upper()} Accuracy')
    plt.xlabel('epoch'); plt.ylabel('acc'); plt.legend(); plt.show()
    return model, hist

# VGG16 먼저 간단 학습
vgg_model, vgg_hist = train_once('vgg16', epochs=10)

# 이어서 ResNet50 간단 학습
res_model, res_hist = train_once('resnet50', epochs=10)

### 3. 모델 성능 평가

- **정량적 평가**
    - 검증 데이터셋을 통해 훈련된 각 모델의 최종 성능을 정량적으로 평가.
    - **평가 지표:** Accuracy, Precision, Recall, F1-Score를 핵심 지표로 사용.

- **결과 제시**
    - VGG16과 ResNet50 모델의 최종 성능을 비교 제시하여 아키텍처별 효율성을 분석.
    - Confusion Matrix을 시각화하여, 모델이 어떤 클래스 간에 혼동을 보이는지 분석.

### 4. 모델 해석 및 정성적 분석

- **정성적 분석**
    - 모델의 의사결정 과정을 시각적으로 해석하기 위해 정성적 분석을 수행.
    - **분석 기법:** Grad-CAM을 사용하여 예측의 근거가 되는 이미지 내 Attention을 시각화.

- **결과 제시**
    - 각 스타일별 예측에 대한 Grad-CAM 히트맵 예시를 제시.

- **분석 목표**
    - 시각화 결과를 통해 모델이 단순히 데이터의 패턴을 암기한 것이 아니라, 각 스타일을 정의하는 유의미한 Semantic 특징을 학습했는지 확인할 계획.